In [1]:
import sys
import os
import numpy as np
import pickle as pk
import copy
import re
from graphviz import Graph
from graphviz import Digraph
sys.path.append('/home/igudich/work/mantid-benchmarks/python/')

In [2]:
from timestamp_tree import *
import dictionary_tree as dtree
from generic_tree import Node, apply_multiple_trees

In [3]:
folder = "/home/igudich/work/SNSPowderReductionResults"
files = os.listdir(folder)
files = [x for x in files if re.match("sns_reduction_[0-9]+_[0-9]{3}\.out", x) ]
data = []
thread_num = []
for file in files:
    print(file)
    thread_num.append(int(re.split("sns_reduction_(.*)_[0-9]{3}\.out", file)[1]))
    header, records = fromFile(os.path.join(folder, file))
    tree = toTrees(records)[0]
    tree = tree.apply(lambda x: [x[0], x[2] - x[1]])
    tree = tree.apply_from_head_childs(lambda x, y: [x[0], x[1] - sum([a[1] for a in y])])
    data.append(tree)

sns_reduction_12_020.out
sns_reduction_8_020.out
sns_reduction_4_020.out
sns_reduction_2_020.out
sns_reduction_1_020.out


In [4]:
def check(infos):
    for info in infos:
        if info[0] != infos[0][0]:
            return False
    return True
apply_multiple_trees(data, check, lambda infos: [infos[0][0], np.polyfit(thread_num, [x[1] for x in infos],1)]).to_dot(displayFunc=lambda x: x[0] + "\n" +  str(x[1]), cond=lambda x: x[1][1] < 0 and x[1][1] > -10)

In [5]:
apply_multiple_trees(data, check, lambda infos: [infos[0][0], [thread_num, [x[1] for x in infos]]]).to_dot(displayFunc=lambda x: x[0] + "\n" +  str(x[1]))